In [ ]:
# ExtractROI - skip dummy scans
'''
To be checked whether it has to be used or not. It just skips some scans from the window.
'''
extract = Node(ExtractROI(t_min=4, t_size=-1),
               output_type='NIFTI_GZ',
               name="extract")


In [ ]:
if (MotionCorrection==1):
    # MCFLIRT - motion correction
    mcflirt = Node(MCFLIRT(mean_vol=True,
                           save_plots=True,
                           output_type='NIFTI_GZ'),
                   name="mcflirt")

In [ ]:
# Smooth - image smoothing
smooth = Node(Smooth(),
              fwhm = FWHM,
              name="smooth")


In [ ]:
# if highpass:
#     highpass = Node(interface=fsl.ImageMaths(suffix='_tempfilt'),
#                           name='highpass')
#     #featpreproc.connect(inputnode, ('highpass', highpass_operand), highpass, 'op_string')
#     featpreproc.connect(meanscale, 'out_file', highpass, 'in_file')

#     """
#     Add back the mean removed by the highpass filter operation as of FSL 5.0.7
#     """
#     meanfunc4 = Node(interface=fsl.ImageMaths(op_string='-Tmean',
#                                                     suffix='_mean'),
#                            name='meanfunc4')

#     featpreproc.connect(meanscale, 'out_file', meanfunc4, 'in_file')
#     addmean = Node(interface=fsl.BinaryMaths(operation='add'),
#                          iterfield=['in_file', 'operand_file'],
#                          name='addmean')
#     featpreproc.connect(highpass, 'out_file', addmean, 'in_file')
#     featpreproc.connect(meanfunc4, 'out_file', addmean, 'operand_file')
#     featpreproc.connect(addmean, 'out_file', outputnode, 'highpassed_files')

In [ ]:
art = Node(ArtifactDetect(norm_threshold=2,
                          zintensity_threshold=3,
                          mask_type='spm_global',
                          parameter_source='FSL',
                          use_differences=[True, False],
                          plot_type='svg'),
           name="art")

In [ ]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['func', 'anat']),
                  name="infosource")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

## Use the following DataSink output substitutions
substitutions = [('_subject_id_', ''),
                 ('_task_name_', '/task-'),
                 ('_fwhm_', 'fwhm-'),
                 ('_roi', ''),
                 ('_mcf', ''),
                 ('_st', ''),
                 ('_flirt', ''),
                 ('.nii_mean_reg', '_mean'),
                 ('.nii.par', '.par'),
                 ]

datasink.inputs.substitutions = substitutions

In [ ]:


# Create a preprocessing workflow
preproc = Workflow(name='preproc')
preproc.base_dir = opj(experiment_dir, working_dir)

# Connect all components of the preprocessing workflow
preproc.connect([(infosource, extract, [('func', 'in_file')]),
                 (extract, mcflirt, [('roi_file', 'in_file')]),
                 (mcflirt, slicetimer, [('out_file', 'in_file')]),

                 (infosource, coregwf, [('anat', 'bet_anat.in_file'),
                                         ('anat', 'coreg_bbr.reference')]),
                 (mcflirt, coregwf, [('mean_img', 'coreg_pre.in_file'),
                                     ('mean_img', 'coreg_bbr.in_file'),
                                     ('mean_img', 'applywarp_mean.in_file')]),
                 (slicetimer, coregwf, [('slice_time_corrected_file', 'applywarp.in_file')]),
                 
                 (coregwf, smooth, [('applywarp.out_file', 'in_files')]),
#                 (inputnode, highpass,[(('highpass', highpass_operand), 'op_string')]),
                 (mcflirt, datasink, [('par_file', 'preproc.@par')]),
                 (smooth, datasink, [('smoothed_files', 'preproc.@smooth')]),
                 
                 (coregwf, datasink, [('applywarp_mean.out_file', 'preproc.@mean')]),

                 (coregwf, art, [('applywarp.out_file', 'realigned_files')]),
                 (mcflirt, art, [('par_file', 'realignment_parameters')]),

                 (coregwf, datasink, [('coreg_bbr.out_matrix_file', 'preproc.@mat_file'),
                                      ('bet_anat.out_file', 'preproc.@brain')]),
                 (art, datasink, [('outlier_files', 'preproc.@outlier_files'),
                                  ('plot_files', 'preproc.@plot_files')]),
                 ])



In [ ]:


# Create preproc output graph
preproc.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph
from IPython.display import Image
Image(filename=opj(preproc.base_dir, 'preproc', 'graph.dot.png'))


In [ ]:
# Visualize the detailed graph
preproc.write_graph(graph2use='flat', format='png', simple_form=True)
Image(filename=opj(preproc.base_dir, 'preproc', 'graph_detailed.dot.png'))


In [ ]:
ab = fsl.ImageStats(op_string='-k %s -p 50', in_file = "/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3577037/session_1/rest_1/rest_linearMNI3mm.nii.gz",
                   mask_file = "/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3577037/session_1/rest_1/rest_linearMNI3mm.nii.gz")
ab.cmdline

In [95]:
import numpy as np
import sys
import json
import Preprocess_network as parallelPreproc
import timeit
start = timeit.default_timer()
# JSONFile = sys.argv[1]
JSONFile = '/home/deepak/Desktop/FConnectivityAnalysis/FConnectivityAnalysisDesign.json'
with open(JSONFile) as JSON:
    try :
        
        data = json.load(JSON)
        AnalysisName = data['Analysis Name']
    #     print(AnalysisName)
        WorkingDir = data['WorkingDir']
        AnalysisParams = data['AnalysisParams']
        
    except :
        print('Unexpected error:', sys.exc_info()[0])
        raise
def run_Preprocessing(AnalysisParams,FunctionalFiles,StructuralFiles):
    B0unwarping = AnalysisParams['B0 Unwarping']
    BETextract = AnalysisParams['BET Brain Extract']
    FWHM = AnalysisParams['FWHM']
    HighPass = AnalysisParams['High Pass']
    MotionCorrection = AnalysisParams['Motion Correction']
    Registration = AnalysisParams['Registration']
    SliceTimeCorrect = AnalysisParams['Slice Time Correct']
    Intensity_Norm = AnalysisParams['Intensity Normalization']
    MelodicICA = AnalysisParams['Melodic ICA']
    PerfusionSubtract = AnalysisParams['Perfusion Subtraction']
    OUTPUT_DIR = AnalysisParams['OutputInfo']['OutDirectory']
    RESULTS_DATASINK = OUTPUT_DIR + "/tmp/featpreproc/datasink/"
      
        
    if HighPass:
        preproc = parallelPreproc.create_parallelfeat_preproc(highpass= HighPass, 
                                    Intensity_Norm = Intensity_Norm,
                                    BETextract = BETextract,
                                    MotionCorrection = MotionCorrection, 
                                    SliceTimeCorrect = SliceTimeCorrect)
        preproc.inputs.inputspec.highpass = 128./(2*2.5)
        preproc.inputs.inputspec.func = FunctionalFiles
        preproc.inputs.inputspec.fwhm = FWHM
        preproc.base_dir = OUTPUT_DIR + '/tmp'
        preproc.write_graph(graph2use='colored', format='png', simple_form=True)
        preproc.run('MultiProc', plugin_args={'n_procs': 4})
        
    else:
        preproc = parallelPreproc.create_parallelfeat_preproc(highpass= HighPass, 
                                    Intensity_Norm = Intensity_Norm,
                                    BETextract = BETextract,
                                    MotionCorrection = MotionCorrection, 
                                    SliceTimeCorrect = SliceTimeCorrect)
        preproc.inputs.inputspec.func = FunctionalFiles
        preproc.inputs.inputspec.fwhm = FWHM
        preproc.base_dir = OUTPUT_DIR + '/tmp'
        preproc.write_graph(graph2use='colored', format='png', simple_form=True)
        preproc.run('MultiProc', plugin_args={'n_procs': 4})
    
    datasink_results=[]
    datasink_results += [each for each in os.listdir(RESULTS_DATASINK) if each.endswith('.json')]
    with open(RESULTS_DATASINK + datasink_results[0]) as JSON:
        datafile = json.load(JSON)
        datafile =datafile[0][1][0][1]
    datasinkouts = []
    datasinkouts += [datafile[i][0] for i in range(len(datafile))]
    
    
    return datasinkouts


ReferenceFile = AnalysisParams['ReferSummary']['ReferImgPath']
ProcessingWay = AnalysisParams['ProcessingType']['ProcessingWay']
Ngroups = AnalysisParams['No of Groups']
if (ProcessingWay==0):
    FunctionaltxtFiles = AnalysisParams['FilesInfo']['FunctionalFilePaths']
    StructuraltxtFiles = AnalysisParams['FilesInfo']['StructuralFilePaths']

    for i in range(0,Ngroups):
        with open(FunctionaltxtFiles[i]) as file:
            FunctionalFiles_in_this_group = [line.strip('\n') for line in file]
        with open(StructuraltxtFiles[i]) as file:
            StructuralFiles_in_this_group = [line.strip('\n') for line in file]
        Preprocessed_Files = run_Preprocessing(AnalysisParams, FunctionalFiles_in_this_group, StructuralFiles_in_this_group)
        print(Preprocessed_Files)

stop = timeit.default_timer()
print("Total time taken for running the program: ", stop - start)



170928-15:51:48,52 workflow INFO:
	 Generated workflow graph: /home/deepak/Desktop/FConnectivityAnalysis/tmp/featpreproc/graph.dot.png (graph2use=colored, simple_form=True).
170928-15:51:48,96 workflow INFO:
	 Workflow featpreproc settings: ['check', 'execution', 'logging']
170928-15:51:48,118 workflow INFO:
	 Running in parallel.
170928-15:51:48,122 workflow INFO:
	 Executing: img2float ID: 0
170928-15:51:48,128 workflow INFO:
	 Adding 12 jobs for mapnode img2float
170928-15:51:48,134 workflow INFO:
	 Executing: _img2float0 ID: 25
170928-15:51:48,136 workflow INFO:
	 [Job finished] jobname: _img2float0 jobid: 25
170928-15:51:48,137 workflow INFO:
	 Executing: _img2float1 ID: 26
170928-15:51:48,139 workflow INFO:
	 [Job finished] jobname: _img2float1 jobid: 26
170928-15:51:48,140 workflow INFO:
	 Executing: _img2float2 ID: 27
170928-15:51:48,142 workflow INFO:
	 [Job finished] jobname: _img2float2 jobid: 27
170928-15:51:48,143 workflow INFO:
	 Executing: _img2float3 ID: 28
170928-15:51

	 [Job finished] jobname: _slicetimer7 jobid: 68
170928-15:51:48,355 workflow INFO:
	 Executing: _slicetimer8 ID: 69
170928-15:51:48,356 workflow INFO:
	 [Job finished] jobname: _slicetimer8 jobid: 69
170928-15:51:48,357 workflow INFO:
	 Executing: _slicetimer9 ID: 70
170928-15:51:48,359 workflow INFO:
	 [Job finished] jobname: _slicetimer9 jobid: 70
170928-15:51:48,360 workflow INFO:
	 Executing: _slicetimer10 ID: 71
170928-15:51:48,361 workflow INFO:
	 [Job finished] jobname: _slicetimer10 jobid: 71
170928-15:51:48,362 workflow INFO:
	 Executing: _slicetimer11 ID: 72
170928-15:51:48,364 workflow INFO:
	 [Job finished] jobname: _slicetimer11 jobid: 72
170928-15:51:48,366 workflow INFO:
	 Executing: slicetimer ID: 3
170928-15:51:48,368 workflow INFO:
	 [Job finished] jobname: slicetimer jobid: 3
170928-15:51:48,369 workflow INFO:
	 Executing: _plot_motion0 ID: 73
170928-15:51:48,370 workflow INFO:
	 [Job finished] jobname: _plot_motion0 jobid: 73
170928-15:51:48,371 workflow INFO:
	 Ex

170928-15:51:48,556 workflow INFO:
	 [Job finished] jobname: _meanfuncmask7 jobid: 116
170928-15:51:48,559 workflow INFO:
	 Executing: _meanfuncmask8 ID: 117
170928-15:51:48,561 workflow INFO:
	 [Job finished] jobname: _meanfuncmask8 jobid: 117
170928-15:51:48,564 workflow INFO:
	 Executing: _meanfuncmask9 ID: 118
170928-15:51:48,565 workflow INFO:
	 [Job finished] jobname: _meanfuncmask9 jobid: 118
170928-15:51:48,567 workflow INFO:
	 Executing: _meanfuncmask10 ID: 119
170928-15:51:48,569 workflow INFO:
	 [Job finished] jobname: _meanfuncmask10 jobid: 119
170928-15:51:48,570 workflow INFO:
	 Executing: _meanfuncmask11 ID: 120
170928-15:51:48,572 workflow INFO:
	 [Job finished] jobname: _meanfuncmask11 jobid: 120
170928-15:51:48,574 workflow INFO:
	 Executing: meanfuncmask ID: 5
170928-15:51:48,576 workflow INFO:
	 [Job finished] jobname: meanfuncmask jobid: 5
170928-15:51:48,579 workflow INFO:
	 Executing: maskfunc ID: 6
170928-15:51:48,588 workflow INFO:
	 Adding 12 jobs for mapnode 

170928-15:51:48,829 workflow INFO:
	 [Job finished] jobname: _dilatemask4 jobid: 161
170928-15:51:48,830 workflow INFO:
	 Executing: _dilatemask5 ID: 162
170928-15:51:48,831 workflow INFO:
	 [Job finished] jobname: _dilatemask5 jobid: 162
170928-15:51:48,832 workflow INFO:
	 Executing: _dilatemask6 ID: 163
170928-15:51:48,833 workflow INFO:
	 [Job finished] jobname: _dilatemask6 jobid: 163
170928-15:51:48,834 workflow INFO:
	 Executing: _dilatemask7 ID: 164
170928-15:51:48,836 workflow INFO:
	 [Job finished] jobname: _dilatemask7 jobid: 164
170928-15:51:48,839 workflow INFO:
	 Executing: _dilatemask8 ID: 165
170928-15:51:48,840 workflow INFO:
	 [Job finished] jobname: _dilatemask8 jobid: 165
170928-15:51:48,841 workflow INFO:
	 Executing: _dilatemask9 ID: 166
170928-15:51:48,842 workflow INFO:
	 [Job finished] jobname: _dilatemask9 jobid: 166
170928-15:51:48,843 workflow INFO:
	 Executing: _dilatemask10 ID: 167
170928-15:51:48,844 workflow INFO:
	 [Job finished] jobname: _dilatemask10 

170928-15:51:49,65 workflow INFO:
	 Executing: _mask5 ID: 210
170928-15:51:49,66 workflow INFO:
	 [Job finished] jobname: _mask5 jobid: 210
170928-15:51:49,67 workflow INFO:
	 Executing: _mask6 ID: 211
170928-15:51:49,68 workflow INFO:
	 [Job finished] jobname: _mask6 jobid: 211
170928-15:51:49,72 workflow INFO:
	 Executing: _mask7 ID: 212
170928-15:51:49,74 workflow INFO:
	 [Job finished] jobname: _mask7 jobid: 212
170928-15:51:49,75 workflow INFO:
	 Executing: _mask8 ID: 213
170928-15:51:49,76 workflow INFO:
	 [Job finished] jobname: _mask8 jobid: 213
170928-15:51:49,78 workflow INFO:
	 Executing: _mask9 ID: 214
170928-15:51:49,80 workflow INFO:
	 [Job finished] jobname: _mask9 jobid: 214
170928-15:51:49,82 workflow INFO:
	 Executing: _mask10 ID: 215
170928-15:51:49,83 workflow INFO:
	 [Job finished] jobname: _mask10 jobid: 215
170928-15:51:49,88 workflow INFO:
	 Executing: _mask11 ID: 216
170928-15:51:49,90 workflow INFO:
	 [Job finished] jobname: _mask11 jobid: 216
170928-15:51:49,

170928-15:51:49,373 workflow INFO:
	 Executing: _meanscale2 ID: 255
170928-15:51:49,375 workflow INFO:
	 [Job finished] jobname: _meanscale2 jobid: 255
170928-15:51:49,376 workflow INFO:
	 Executing: _meanscale3 ID: 256
170928-15:51:49,378 workflow INFO:
	 [Job finished] jobname: _meanscale3 jobid: 256
170928-15:51:49,383 workflow INFO:
	 Executing: _meanscale4 ID: 257
170928-15:51:49,385 workflow INFO:
	 [Job finished] jobname: _meanscale4 jobid: 257
170928-15:51:49,386 workflow INFO:
	 Executing: _meanscale5 ID: 258
170928-15:51:49,388 workflow INFO:
	 [Job finished] jobname: _meanscale5 jobid: 258
170928-15:51:49,390 workflow INFO:
	 Executing: _meanscale6 ID: 259
170928-15:51:49,392 workflow INFO:
	 [Job finished] jobname: _meanscale6 jobid: 259
170928-15:51:49,394 workflow INFO:
	 Executing: _meanscale7 ID: 260
170928-15:51:49,396 workflow INFO:
	 [Job finished] jobname: _meanscale7 jobid: 260
170928-15:51:49,402 workflow INFO:
	 Executing: _meanscale8 ID: 261
170928-15:51:49,403 

<bound method Graph.size of <networkx.classes.digraph.DiGraph object at 0x7f7e8e16feb8>>

In [57]:
import os
import json
import nipype.interfaces.utility as util
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node, MapNode

from nipype.interfaces import fsl
import nipype.workflows.fmri.fsl.preprocess as Preprocessor

In [58]:
Reg_WorkFlow = Preprocessor.create_reg_workflow()

In [61]:
meanfunc = fsl.ImageMaths(op_string='-Tmean',
                                                    suffix='_mean', 
            in_file = '/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3577037/session_1/rest_1/rest_linearMNI3mm.nii.gz')
    

#Reg_WorkFlow.connect(meanfunc, 'out_file', Reg_WorkFlow.inputs.inputspec, 'mean_image')
Reg_WorkFlow.inputs.inputspec.mean_image = meanfunc.out_file

AttributeError: 'ImageMaths' object has no attribute 'out_file'

'/home/deepak/Desktop/funcConn/funcConnGUI/Scripts/foo_maths.nii.gz'

In [154]:
def reg_workflow(name = 'registration'):
    """Create a FEAT preprocessing workflow
    Parameters
    ----------
    ::
        name : name of workflow (default: 'registration')
    Inputs::
        inputspec.source_files : files (filename or list of filenames to register)
        inputspec.mean_image : reference image to use
        inputspec.anatomical_image : anatomical image to coregister to
        inputspec.target_image : registration target
    Outputs::
        outputspec.func2anat_transform : FLIRT transform
        outputspec.anat2target_transform : FLIRT+FNIRT transform
        outputspec.transformed_files : transformed files in target space
        outputspec.transformed_mean : mean image in target space
    Example
    -------
    """

    register = Workflow(name=name)

    inputnode = Node(interface=util.IdentityInterface(fields=['source_files',
                                                                 'anatomical_image',
                                                                 'target_image']),
                        name='inputspec')
    outputnode = Node(interface=util.IdentityInterface(fields=['func2anat_transform',
                                                                  'anat2target_transform',
                                                                  'transformed_files',
                                                                  'transformed_mean',
                                                                  ]),
                         name='outputspec')
    meanfunc = Node(fsl.ImageMaths(op_string='-Tmean',suffix='_mean'), name = 'meanfunc')
    register.connect(inputnode, 'source_files', meanfunc, 'in_file')
    """
    Estimate the tissue classes from the anatomical image. But use spm's segment
    as FSL appears to be breaking.
    """

    stripper = Node(fsl.BET(), name='stripper')
    register.connect(inputnode, 'anatomical_image', stripper, 'in_file')
    fast = Node(fsl.FAST(), name='fast')
    register.connect(stripper, 'out_file', fast, 'in_files')

    """
    Binarize the segmentation
    """

    binarize = Node(fsl.ImageMaths(op_string='-nan -thr 0.5 -bin'),
                       name='binarize')
    pickindex = lambda x, i: x[i]
    register.connect(fast, ('partial_volume_files', pickindex, 2),
                     binarize, 'in_file')

    """
    Calculate rigid transform from mean image to anatomical image
    """

    mean2anat = Node(fsl.FLIRT(), name='mean2anat')
    mean2anat.inputs.dof = 6
    register.connect(meanfunc,'out_file', mean2anat, 'in_file')
    register.connect(stripper, 'out_file', mean2anat, 'reference')

    """
    Now use bbr cost function to improve the transform
    """

    mean2anatbbr = Node(fsl.FLIRT(), name='mean2anatbbr')
    mean2anatbbr.inputs.dof = 6
    mean2anatbbr.inputs.cost = 'bbr'
    mean2anatbbr.inputs.schedule = os.path.join(os.getenv('FSLDIR'),
                                                'etc/flirtsch/bbr.sch')
    register.connect(meanfunc,'out_file', mean2anatbbr, 'in_file')
    register.connect(binarize, 'out_file', mean2anatbbr, 'wm_seg')
    register.connect(inputnode, 'anatomical_image', mean2anatbbr, 'reference')
    register.connect(mean2anat, 'out_matrix_file',
                     mean2anatbbr, 'in_matrix_file')

    """
    Calculate affine transform from anatomical to target
    """

    anat2target_affine = Node(fsl.FLIRT(), name='anat2target_linear')
    anat2target_affine.inputs.searchr_x = [-180, 180]
    anat2target_affine.inputs.searchr_y = [-180, 180]
    anat2target_affine.inputs.searchr_z = [-180, 180]
    register.connect(stripper, 'out_file', anat2target_affine, 'in_file')
    register.connect(inputnode, 'target_image',
                     anat2target_affine, 'reference')


#     register.connect(anat2target_affine, 'out_matrix_file',
#                      anat2target_nonlinear, 'affine_file')
#     register.connect(inputnode, 'anatomical_image',
#                      anat2target_nonlinear, 'in_file')
#     register.connect(inputnode, 'config_file',
#                      anat2target_nonlinear, 'config_file')
#     register.connect(inputnode, 'target_image',
#                      anat2target_nonlinear, 'ref_file')

    """
    Transform the mean image. First to anatomical and then to target
    """

    warpmean = Node(fsl.ApplyWarp(interp='spline'), name='warpmean')
    register.connect(meanfunc,'out_file', warpmean, 'in_file')
    register.connect(mean2anatbbr, 'out_matrix_file', warpmean, 'premat')
    register.connect(inputnode, 'target_image', warpmean, 'ref_file')
#     register.connect(anat2target_nonlinear, 'fieldcoeff_file',
#                      warpmean, 'field_file')
    register.connect(anat2target_affine, 'out_matrix_file',
                     warpmean, 'postmat')
    
    """
    Transform the remaining images. First to anatomical and then to target
    """

    warpall = MapNode(fsl.ApplyWarp(interp='spline'),
                         iterfield=['in_file'],
                         nested=True,
                         name='warpall')
    datasink = Node(interface=DataSink(), name="datasink")
    register.connect(inputnode, 'source_files', warpall, 'in_file')
    register.connect(mean2anatbbr, 'out_matrix_file', warpall, 'premat')
    register.connect(inputnode, 'target_image', warpall, 'ref_file')
#     register.connect(anat2target_nonlinear, 'fieldcoeff_file',
#                      warpall, 'field_file')
    register.connect(anat2target_affine, 'out_matrix_file',
                     warpall, 'postmat')

    """
    Assign all the output files
    """

    register.connect(warpmean, 'out_file', outputnode, 'transformed_mean')
    register.connect(warpall, 'out_file', outputnode, 'transformed_files')
    register.connect(mean2anatbbr, 'out_matrix_file',
                     outputnode, 'func2anat_transform')
#     register.connect(anat2target_nonlinear, 'fieldcoeff_file',
#                      outputnode, 'anat2target_transform')
    register.connect(anat2target_affine, 'out_matrix_file',
                     outputnode, 'anat2target_transform')
    register.connect(warpall, 'out_file', datasink,'out_file') 

    
    return register

In [156]:
Reg_WorkFlow = reg_workflow()
Reg_WorkFlow.inputs.inputspec.source_files = '/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/1065809/session_1/rest_1/rest_linearMNI3mm.nii.gz'

Reg_WorkFlow.inputs.inputspec.anatomical_image = '/usr/share/fsl/5.0/data/standard/MNI152_T1_2mm_brain.nii.gz'
Reg_WorkFlow.inputs.inputspec.target_image = '/usr/share/fsl/5.0/data/standard/MNI152_T1_2mm_brain.nii.gz'

Reg_WorkFlow.base_dir = os.getcwd() + '/tmp'


regoutputs = Reg_WorkFlow.run()

170928-19:10:21,689 workflow INFO:
	 Workflow registration settings: ['check', 'execution', 'logging']
170928-19:10:21,696 workflow INFO:
	 Running serially.
170928-19:10:21,697 workflow INFO:
	 Executing node stripper in dir: /home/deepak/Desktop/funcConn/funcConnGUI/Scripts/tmp/registration/stripper
170928-19:10:21,698 workflow INFO:
170928-19:10:21,699 workflow INFO:
	 Executing node fast in dir: /home/deepak/Desktop/funcConn/funcConnGUI/Scripts/tmp/registration/fast
170928-19:10:21,700 workflow INFO:
170928-19:10:21,703 workflow INFO:
	 Executing node binarize in dir: /home/deepak/Desktop/funcConn/funcConnGUI/Scripts/tmp/registration/binarize
170928-19:10:21,704 workflow INFO:
170928-19:10:21,705 workflow INFO:
	 Executing node anat2target_linear in dir: /home/deepak/Desktop/funcConn/funcConnGUI/Scripts/tmp/registration/anat2target_linear
170928-19:10:21,706 workflow INFO:
170928-19:10:21,707 workflow INFO:
	 Executing node meanfunc in dir: /home/deepak/Desktop/funcConn/funcConnGUI

In [160]:
RESULTS_REG_DATASINK = '/home/deepak/Desktop/funcConn/funcConnGUI/Scripts/tmp/registration/datasink/'
datasink_results=[]
datasink_results += [each for each in os.listdir(RESULTS_REG_DATASINK) if each.endswith('.json')]
with open(RESULTS_REG_DATASINK + datasink_results[0]) as JSON:
    datafile = json.load(JSON)
    datafile =datafile[0][1][0][1]
datasinkouts = []
datasinkouts += [datafile[i][0] for i in range(len(datafile))]

In [163]:
str(2)

'2'

In [132]:
def reg_workflow(name = 'registration'):
    """Create a FEAT preprocessing workflow
    Parameters
    ----------
    ::
        name : name of workflow (default: 'registration')
    Inputs::
        inputspec.source_files : files (filename or list of filenames to register)
        inputspec.mean_image : reference image to use
        inputspec.anatomical_image : anatomical image to coregister to
        inputspec.target_image : registration target
    Outputs::
        outputspec.func2anat_transform : FLIRT transform
        outputspec.anat2target_transform : FLIRT+FNIRT transform
        outputspec.transformed_files : transformed files in target space
        outputspec.transformed_mean : mean image in target space
    Example
    -------
    """

    register = Workflow(name=name)

    inputnode = Node(interface=util.IdentityInterface(fields=['source_files',
                                                                 'anatomical_image',
                                                                 'target_image']),
                        name='inputspec')
    outputnode = Node(interface=util.IdentityInterface(fields=['func2anat_transform',
                                                                  'anat2target_transform',
                                                                  'transformed_files',
                                                                  'transformed_mean',
                                                                  ]),
                         name='outputspec')
    meanfunc = MapNode(fsl.ImageMaths(op_string='-Tmean',suffix='_mean'), 
                       iterfield = ['in_file'], 
                       name = 'meanfunc')
    register.connect(inputnode, 'source_files', meanfunc, 'in_file')
    """
    Estimate the tissue classes from the anatomical image. But use spm's segment
    as FSL appears to be breaking.
    """

    stripper = MapNode(fsl.BET(), iterfield = ['in_file'],
                       name='stripper')
    register.connect(inputnode, 'anatomical_image', stripper, 'in_file')
    fast = MapNode(fsl.FAST(), iterfield = ['in_files'], name='fast')
    register.connect(stripper, 'out_file', fast, 'in_files')

    """
    Binarize the segmentation
    """

    binarize = MapNode(fsl.ImageMaths(op_string='-nan -thr 0.5 -bin'),
                       iterfield = ['in_file'],
                       name='binarize')
    pickindex = lambda x, i: x[i]
    register.connect(fast, ('partial_volume_files', pickindex, 2),
                     binarize, 'in_file')

    """
    Calculate rigid transform from mean image to anatomical image
    """

    mean2anat = MapNode(fsl.FLIRT(), 
                        iterfield = ['in_file','reference'],
                        name='mean2anat')
    mean2anat.inputs.dof = 6
    register.connect(meanfunc,'out_file', mean2anat, 'in_file')
    register.connect(stripper, 'out_file', mean2anat, 'reference')

    """
    Now use bbr cost function to improve the transform
    """

    mean2anatbbr = MapNode(fsl.FLIRT(), 
                        iterfield = ['in_file','wm_seg','reference','in_matrix_file'],
                        name='mean2anatbbr')
    mean2anatbbr.inputs.dof = 6
    mean2anatbbr.inputs.cost = 'bbr'
    mean2anatbbr.inputs.schedule = os.path.join(os.getenv('FSLDIR'),
                                                'etc/flirtsch/bbr.sch')
    register.connect(meanfunc,'out_file', mean2anatbbr, 'in_file')
    register.connect(binarize, 'out_file', mean2anatbbr, 'wm_seg')
    register.connect(inputnode, 'anatomical_image', mean2anatbbr, 'reference')
    register.connect(mean2anat, 'out_matrix_file',
                     mean2anatbbr, 'in_matrix_file')

    """
    Calculate affine transform from anatomical to target
    """

    anat2target_affine = MapNode(fsl.FLIRT(), iterfield = ['in_file','reference'],
                                 name='anat2target_linear')
    anat2target_affine.inputs.searchr_x = [-180, 180]
    anat2target_affine.inputs.searchr_y = [-180, 180]
    anat2target_affine.inputs.searchr_z = [-180, 180]
    register.connect(stripper, 'out_file', anat2target_affine, 'in_file')
    register.connect(inputnode, 'target_image',
                     anat2target_affine, 'reference')


#     register.connect(anat2target_affine, 'out_matrix_file',
#                      anat2target_nonlinear, 'affine_file')
#     register.connect(inputnode, 'anatomical_image',
#                      anat2target_nonlinear, 'in_file')
#     register.connect(inputnode, 'config_file',
#                      anat2target_nonlinear, 'config_file')
#     register.connect(inputnode, 'target_image',
#                      anat2target_nonlinear, 'ref_file')

    """
    Transform the mean image. First to anatomical and then to target
    """

    warpmean = MapNode(fsl.ApplyWarp(interp='spline'), 
                       iterfield = ['in_file','premat','ref_file','postmat'],
                       name='warpmean')
    register.connect(meanfunc,'out_file', warpmean, 'in_file')
    register.connect(mean2anatbbr, 'out_matrix_file', warpmean, 'premat')
    register.connect(inputnode, 'target_image', warpmean, 'ref_file')
#     register.connect(anat2target_nonlinear, 'fieldcoeff_file',
#                      warpmean, 'field_file')
    register.connect(anat2target_affine, 'out_matrix_file',
                     warpmean, 'postmat')
    
    """
    Transform the remaining images. First to anatomical and then to target
    """

    warpall = MapNode(fsl.ApplyWarp(interp='spline'),
                         iterfield=['in_file',],
                         nested=True,
                         name='warpall')
    register.connect(inputnode, 'source_files', warpall, 'in_file')
    register.connect(mean2anatbbr, 'out_matrix_file', warpall, 'premat')
    register.connect(inputnode, 'target_image', warpall, 'ref_file')
#     register.connect(anat2target_nonlinear, 'fieldcoeff_file',
#                      warpall, 'field_file')
    register.connect(anat2target_affine, 'out_matrix_file',
                     warpall, 'postmat')

    """
    Assign all the output files
    """

    register.connect(warpmean, 'out_file', outputnode, 'transformed_mean')
    register.connect(warpall, 'out_file', outputnode, 'transformed_files')
    register.connect(mean2anatbbr, 'out_matrix_file',
                     outputnode, 'func2anat_transform')
#     register.connect(anat2target_nonlinear, 'fieldcoeff_file',
#                      outputnode, 'anat2target_transform')
    register.connect(anat2target_affine, 'out_matrix_file',
                     outputnode, 'anat2target_transform')
    

    
    return register

Wraps command **flirt**

Use FSL FLIRT for coregistration.

For complete details, see the `FLIRT Documentation.
<https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FLIRT>`_

To print out the command line help, use:
    fsl.FLIRT().inputs_help()

Examples
--------
>>> from nipype.interfaces import fsl
>>> from nipype.testing import example_data
>>> flt = fsl.FLIRT(bins=640, cost_func='mutualinfo')
>>> flt.inputs.in_file = 'structural.nii'
>>> flt.inputs.reference = 'mni.nii'
>>> flt.inputs.output_type = "NIFTI_GZ"
>>> flt.cmdline # doctest: +ELLIPSIS +ALLOW_UNICODE
'flirt -in structural.nii -ref mni.nii -out structural_flirt.nii.gz -omat structural_flirt.mat -bins 640 -searchcost mutualinfo'
>>> res = flt.run() #doctest: +SKIP

Inputs::

	[Mandatory]
	in_file: (an existing file name)
		input file
		flag: -in %s, position: 0
	reference: (an existing file name)
		reference file
		flag: -ref %s, position: 1

	[Optional]
	angle_rep: ('quaternion' or 'euler')
		representation of rotation angles
		flag: 

In [133]:
fsl.FNIRT.help()

Wraps command **fnirt**

Use FSL FNIRT for non-linear registration.

For complete details, see the `FNIRT Documentation.
<https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FNIRT>`_

Examples
--------
>>> from nipype.interfaces import fsl
>>> from nipype.testing import example_data
>>> fnt = fsl.FNIRT(affine_file=example_data('trans.mat'))
>>> res = fnt.run(ref_file=example_data('mni.nii', in_file=example_data('structural.nii')) #doctest: +SKIP

T1 -> Mni153

>>> from nipype.interfaces import fsl
>>> fnirt_mprage = fsl.FNIRT()
>>> fnirt_mprage.inputs.in_fwhm = [8, 4, 2, 2]
>>> fnirt_mprage.inputs.subsampling_scheme = [4, 2, 1, 1]

Specify the resolution of the warps

>>> fnirt_mprage.inputs.warp_resolution = (6, 6, 6)
>>> res = fnirt_mprage.run(in_file='structural.nii', ref_file='mni.nii', warped_file='warped.nii', fieldcoeff_file='fieldcoeff.nii')#doctest: +SKIP

We can check the command line and confirm that it's what we expect.

>>> fnirt_mprage.cmdline  #doctest: +SKIP
'fnirt --cout=fieldcoeff

In [134]:
fsl.ApplyWarp.help()

Wraps command **applywarp**

Use FSL's applywarp to apply the results of a FNIRT registration

Examples
--------
>>> from nipype.interfaces import fsl
>>> from nipype.testing import example_data
>>> aw = fsl.ApplyWarp()
>>> aw.inputs.in_file = example_data('structural.nii')
>>> aw.inputs.ref_file = example_data('mni.nii')
>>> aw.inputs.field_file = 'my_coefficients_filed.nii' #doctest: +SKIP
>>> res = aw.run() #doctest: +SKIP

Inputs::

	[Mandatory]
	in_file: (an existing file name)
		image to be warped
		flag: --in=%s, position: 0
	ref_file: (an existing file name)
		reference image
		flag: --ref=%s, position: 1

	[Optional]
	abswarp: (a boolean)
		treat warp field as absolute: x' = w(x)
		flag: --abs
		mutually_exclusive: relwarp
	args: (a unicode string)
		Additional parameters to the command
		flag: %s
	datatype: ('char' or 'short' or 'int' or 'float' or 'double')
		Force output data type [char short int float double].
		flag: --datatype=%s
	environ: (a dictionary with keys which a

/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3999447/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3283638/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/2154093/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/1065809/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/4222184/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3334426/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/4098356/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3003775/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/1915879/session_1/rest_1/rest_linearMNI3mm.nii.gz